## CoT (Chain of Thought) Prompting Baseline Notebook
### This is the notebook version of CoT.py + CoT.sh. Use this if you want to go through the step-by-step outputs and definitions for the CoT 

In [1]:
import re
import ast
import base64
import requests
import json, os
import pandas as pd
from pprint import pprint
import sys
from pathlib import Path

# set parent directory to address relative imports
directory = Path(os.getcwd()).absolute()
sys.path.append(str(directory.parent.parent.parent))

# import environment
from AI2Thor.env_new import AI2ThorEnv
from AI2Thor.base_env import convert_dict_to_string
from AI2Thor.object_actions import get_closest_feasible_action, get_closest_object_id
from AI2Thor.baselines.utils import Logger, AutoConfig
os.environ["TOKENIZERS_PARALLELISM"] = "true"

# save a json file with your openai api key in your
# home folder as {"my_openai_api_key": "INSERT API HERE"}
with open(os.path.expanduser("~") + "/openai_key.json") as json_file:
    key = json.load(json_file)
    api_key = key["my_openai_api_key"]
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}

/Users/marinatenhave/miniconda3/envs/uropdinamopy39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# initialize environment parameters
class Config:
    def __init__(self):
        self.num_agents = 2
        self.scene = "FloorPlan2"
        self.scene_name = "FloorPlan2"
        self.model = "gpt-4"
        self.horizon = 30
        self.use_langchain = False
        self.use_strict_format = True
        self.use_obs_summariser = False
        self.use_act_summariser = False
        self.use_action_failure = True
        self.use_shared_subtask = True
        self.use_separate_subtask = False
        self.use_future_message = True
        self.forceAction = False
        self.use_memory = True
        self.use_plan = True
        self.use_separate_memory = False
        self.use_shared_memory = True
        self.temperature = 0.7

config = Config()
env = AI2ThorEnv(config)
env.controller.stop()

In [3]:
# CoT PROMPT
agent_name = ["Alice", "Bob"]
PROMPT = f"""You are an excellent planner who is tasked with helping 2 embodied robots named {agent_name[0]} and {agent_name[1]} carry out a task. Both robots have a partially observable view of the environment. Hence they have to explore around in the environment to do the task.
They can perform the following actions: ["navigate to object <object_id>", "rotate in <rotation> direction", "pick up object <object_id>", "put object on <receptacle_id>", "open object <object_id>", "close object <object_id>", "slice object <object_id>", “toggle object <object_id> on”, “toggle object <object_id> off”, "clean object <object_id>", "look up by angle <angle>", "look down by angle <angle>", “move in <translation> direction", "stay idle", "Done"]
Here "Done" is used when all the robots have completed the main task. Only use it when you think all the subtasks are complete.
"stay idle" is used when you want the robot to stay idle for one time step. This could be used to wait for the other robot to complete its subtask. Use it only when you think it is necessary.
Here <rotation> can be one of ["Right", "Left"].
Here <angle> is the angle in degrees and can only be one of [30, 60, 90, 120, 150, 180].
Here <translation> can be one of ["Ahead", "Back", "Left", "Right”].

You need to suggest the action that each robot should take at the current time step.

### Important Notes ###
* The robots can hold only one object at a time.
For example: If {agent_name} is holding an apple, it cannot pick up another object until it puts the apple down.
* Even if the robot can see objects, it might not be able to interact with them if they are too far away. Hence you will need to make the robot navigate closer to the objects they want to interact with.
For example: An action like "pick up <object_id>" is feasible only if robot can see the object and is close enough to it. So you will have to navigate closer to it before you can pick it up.
In some scenarios, the agents might not see the objects that they want to interact with. In such cases, you will have to make the robot explore the environment to find the object.
In such scenarios you can use actions to rotate in place or look up / down or navigate to explore the environment.
* If you open an object, please ensure that you close it before you navigate to a different place.
* Opening object like drawers, cabinets, fridge can block the path of the robot. So open objects only when you think it is necessary.

### INPUT FORMAT ###
* You will get a description of the task robots are supposed to do.
* You will get an image of the environment at the current time step from {agent_name[0]}'s perspective and {agent_name[1]}'s perspective as the observation input. To help you with detecting objects in the image, you will also get a list objects each agent is able to see in the environment. Here the objects are named as "<object_name>_<object_id>". 
* You will get a trace of the steps taken by the robots and the actions they took at each time step and whether it was successful or not.

### OUTPUT FORMAT ###
You are supposed to FIRST reason through the situation logically and step by step, then suggest the action each robot is supposed to take at the current time step.
In your output, do not have any extra text or content outside of the python dictionary as below. 
Your output should ONLY be in the form of a python dictionary as shown below:
{{"reason":"Reasoning for action plan....", "{agent_name[0]}": "action to be taken by {agent_name[0]}", "{agent_name[1]}": "action to be taken by {agent_name[1]}"}}
Put all of your reasoning inside of the "reason" key of the dictonary. Do NOT put any text, spaces, or enter keys (i.e. "/n") outside of it. 

For example: If you think {agent_name[0]} should pick up an apple and {agent_name[1]} should navigate to the fridge, you will have to give the output as:
{{"reason": "since the subtask list is empty, the robots need to transport the apple to the fridge", "{agent_name[0]}": "pick up apple", "{agent_name[1]}": "navigate to fridge"}}

Let's think step by step, but make sure to put all of your reasoning inside of the "reason" key of the dictionary!

* NOTE: DO NOT OUTPUT ANYTHING EXTRA OTHER THAN WHAT HAS BEEN SPECIFIED
"""


In [4]:
def get_action_llm_input(env):
    """
    Returns the input to the subtask LLM
    ### INPUT FORMAT ###
    {{Task: description of the task the robots are supposed to do,
    {agent_name[i]}'s observation: list of objects the {agent_name[0]} is observing}}
    """
    # extract the agent_name's observations based on how many agents there are
    llm_input_feats = []
    for i in range(env.num_agents):
        agent_name = env.agent_names[i]
        llm_input_feats.extend([agent_name + "'s observation", ])
    return dict((k, env.input_dict[k]) for k in llm_input_feats)

In [5]:
def convert_obs_list_str(string_list):
    my_list = ast.literal_eval(string_list)
     #my_list = ['Stove_1', 'Drawer_1', 'Cabinet_1', 'Cabinet_4']
    # Join all elements in the list with commas
    result = ', '.join(my_list)
    return result

In [6]:
def prepare_prompt(user_prompt:str, step_num:int):
    """module_name: str 
        choose from planner, verifier, action
    """
    system_prompt = PROMPT
    user_prompt += f"\nStep {step_num}:\n"
    out_dict = get_action_llm_input(env)
    for i in range(env.num_agents):
        agent = env.agent_names[i]
        obs = convert_obs_list_str(out_dict[f"{agent}'s observation"])
        user_prompt += f"{agent} observes {obs}\n"
    
    # user_prompt = convert_dict_to_string(get_action_llm_input(env))
    return system_prompt, user_prompt

In [7]:
def encode_image(image_path:str):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
def get_action_llm_input(env):
    """
    Returns the input to the subtask LLM
    ### INPUT FORMAT ###
    {{Task: description of the task the robots are supposed to do,
    {agent_name[i]}'s observation: list of objects the {agent_name[0]} is observing}}
    """
    # extract the agent_name's observations based on how many agents there are
    llm_input_feats = ["Task"]
    for i in range(env.num_agents):
        agent_name = env.agent_names[i]
        llm_input_feats.extend([agent_name + "'s observation", ])
    return dict((k, env.input_dict[k]) for k in llm_input_feats)

def prepare_payload(user_prompt, step_num:int):
    """# payload consists of 
    * the image from each agent's perspective
    * the system prompt (which is constant)
    * the user prompt (which changes based on the state)
    This is then sent to the openai api to get the response (action or plan or verification of the plan)
    """
    system_prompt, user_prompt = prepare_prompt(user_prompt, step_num)
    base64_image = []
    image_path = env.get_frame(0)
    base64_image.append(encode_image(image_path))
    image_path = env.get_frame(1)
    base64_image.append(encode_image(image_path))
    payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
            {
                "role": "system",
                "content": [
                    {"type": "text", "text": system_prompt},
                ],
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image[0]}"},
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image[1]}"},
                    },
                ],
            }
        ],
        "max_tokens": 1000,
        "temperature": config.temperature,
    }
    return payload, user_prompt


def get_action(response):
    response_dict = response.json()
    # convert the string to a dict
    # json_acceptable_string = response_dict["choices"][0]["message"]["content"].replace("'", "\"").replace("\n", "").replace("json", "").replace("`", "")
    try:
        output = response_dict["choices"][0]["message"]["content"]
    except:
        pprint(f"error: choices not in response_dict\n {response_dict}\n")
    dict_match = re.search(r'\{.*\}', output, re.DOTALL)

    if dict_match:
        # Extract the dictionary from the matched string
        return json.loads(dict_match.group())
    else:
        # parsing error
        pprint(f"response_dict\n {response_dict}\n")
        pprint(f"output\n {output}\n")
        pprint(f"dict_match\n {dict_match}\n")

def get_gpt_response(user_prompt, step_num:int):
    payload, user_prompt = prepare_payload(user_prompt, step_num)
    response = requests.post(
    "https://api.openai.com/v1/chat/completions", headers=headers, json=payload
)
    return response, user_prompt


#  "look up by angle <angle>", "look down by angle <angle>"
def action_checker(actions):
    """
    Get closest valid action

    The action output from the model is in natural language.
    This function will find the env feasible action which has the closest embedding 
    to the natural language action output from the model.
    Eg: "pick up the apple" -> "PickupObject(Apple_1)"
    """
    checked_actions = []
    for act in actions:
        act = get_closest_feasible_action(act)
        action_type = act.split("(")[0]
        if action_type in ['PickupObject', 'PutObject', 'OpenObject', 'CloseObject', 'SliceObject', 'NavigateTo', 'ToggleObjectOn', 'ToggleObjectOff', 'CleanObject']:
            act = get_closest_object_id(act, env.object_dict)
        checked_actions.append(act)
    return checked_actions

In [8]:
def prepare_prompt_post_action(user_prompt, action, action_successes):
    for i in range(env.num_agents):
        agent = env.agent_names[i]
        user_prompt += f"{agent}'s action was {action[i]} and it was {'successful' if action_successes[i] else 'unsuccessful'}\n"
    return user_prompt

In [9]:
from AI2Thor.baselines.utils.auto_config import AutoConfig

auto=AutoConfig()
amt_tasks=auto.get_amt_tasks()
for task_index in range(amt_tasks):

    auto.set_task(task_index)
    amt_floorplans=auto.get_amt_floorplans(task_index)

    for fp_index in range(amt_floorplans):

        auto.set_floorplan(fp_index)
        timeout=auto.get_task_timeout()

        max_retries = 3
        retries = 0

        while retries < max_retries:
            try: # sometimes rare errors happen, so give each task and floorplan a few tries

                env = AI2ThorEnv(auto.config())
                task = auto.task_string()
                d = env.reset(task=task)

                user_prompt = f"Task: {task}"

                logger=Logger('CoT', env) 
                for step_num in range(1, timeout+1):
                    response, user_prompt = get_gpt_response(user_prompt, step_num)
                    outdict = get_action(response)
                    # get closest feasible action
                    action_texts = [outdict[agent_name[0]], outdict[agent_name[1]]]
                    action = action_checker(action_texts)
                    # execute action in environment
                    d, action_successes = env.step(action)
                    # update user prompt with action taken and its success
                    user_prompt = prepare_prompt_post_action(user_prompt, action_texts, action_successes)
                    # append to dataframe
                    coverage = env.checker.get_coverage()
                    transport_rate = env.checker.get_transport_rate()
                    finished = env.checker.check_success()
                    # logging
                    logger.log_step(step=step_num, preaction=action_texts, action=action, success=action_successes, coverage=coverage, transport_rate=transport_rate, finished=finished)
                    print('_'*50)
                    print(f"Step {step_num}")
                    print(f"Completed Subtasks: ")
                    print("\n".join(env.checker.subtasks_completed))
                    # if the model outputs "Done" for both agents, break
                    if all(status == 'Done' for status in action):
                        break

                env.controller.stop()
                break
            except Exception as e:
                print(f"Error occurred: {e}. Retrying...")
                env.controller.stop()
                retries += 1
                if retries >= max_retries:
                    print("Max retries reached. Moving to next floorplan or task.")
                    break

Setting Config Task to 0, with task name 'Put the bread, lettuce, and tomato in the fridge'. Floorplan unknown.
Setting Config FloorPlan to 0, with name 'FloorPlan1'
__________________________________________________
Pre-initializing environment with closest task: 1_put_bread_lettuce_tomato_fridge
__________________________________________________
__________________________________________________
Subtasks to complete:
NavigateTo(Bread)
PickupObject(Bread)
NavigateTo(Fridge, Bread)
PutObject(Fridge, Bread)
NavigateTo(Tomato)
PickupObject(Tomato)
NavigateTo(Fridge, Tomato)
PutObject(Fridge, Tomato)
NavigateTo(Lettuce)
PickupObject(Lettuce)
NavigateTo(Fridge, Lettuce)
PutObject(Fridge, Lettuce)
OpenObject(Fridge)
CloseObject(Fridge)
__________________________________________________
Preinitializing the environment
__________________________________________________


/Users/marinatenhave/uropmultiagent/SayCanPlaygroundOrig/AI2Thor/baselines/utils/logging.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, row])


__________________________________________________
Step 1
Completed Subtasks: 



KeyboardInterrupt: 

In [ ]:
pprint(logger.summarize())

In [10]:
import pandas as pd

# Define the append_row function to accept df as an argument
def append_row(df, step, action, success, coverage, transport_rate):
    row = pd.DataFrame([[step, action, success, coverage, transport_rate]], columns=['Step', 'Action', 'Success', 'Coverage', 'Transport Rate'])
    df = pd.concat([df, row])
    return df

def old_method():
    # Initialize an empty DataFrame
    df = pd.DataFrame(columns=['Step', 'Action', 'Success', 'Coverage', 'Transport Rate'])

    env = AI2ThorEnv(config)
    task = "Put the bread, lettuce, and tomato in the fridge"
    d = env.reset(task=task)
    user_prompt = f"Task: {task}"

    for step_num in range(1, config.horizon+1):
        response, user_prompt = get_gpt_response(user_prompt, step_num)
        outdict = get_action(response)
        print("Outdict:", outdict)
        # get closest feasible action
        action_texts = [outdict[agent_name[0]], outdict[agent_name[1]]]
        action = action_checker(action_texts)
        # execute action in environment
        d, action_successes = env.step(action)
        # update user prompt with action taken and its success
        user_prompt = prepare_prompt_post_action(user_prompt, action_texts, action_successes)
        # append to dataframe
        coverage = env.checker.get_coverage()
        transport_rate = env.checker.get_transport_rate()
        # Pass df as an argument to append_row and assign the returned DataFrame back to df
        df = append_row(df, step_num, action, action_successes, coverage, transport_rate)
        # if the model outputs "Done" for both agents, break
        print('_'*50)
        print(f"Step {step_num}")
        print(f"Completed Subtasks: ")
        print("\n".join(env.checker.subtasks_completed))
        if all(status == 'Done' for status in action):
            break
 
    # Return the DataFrame after all iterations
    return df

# Call the old_method function and assign its return value to df
df = old_method()


__________________________________________________
Pre-initializing environment with closest task: 1_put_bread_lettuce_tomato_fridge
__________________________________________________
__________________________________________________
Subtasks to complete:
NavigateTo(Bread)
PickupObject(Bread)
NavigateTo(Fridge, Bread)
PutObject(Fridge, Bread)
NavigateTo(Tomato)
PickupObject(Tomato)
NavigateTo(Fridge, Tomato)
PutObject(Fridge, Tomato)
NavigateTo(Lettuce)
PickupObject(Lettuce)
NavigateTo(Fridge, Lettuce)
PutObject(Fridge, Lettuce)
OpenObject(Fridge)
CloseObject(Fridge)
__________________________________________________
Preinitializing the environment
__________________________________________________
Outdict: {'reason': "To begin the task, Alice should pick up the bread since it is visible and within reach. Bob's view is limited and the fridge is not fully in sight, but given Bob is facing the fridge, Bob should rotate to the right to fully face the fridge, which will likely make it pos

/var/folders/gb/9_26mfms6j99m2rwt73bt0v00000gn/T/ipykernel_68375/1654365176.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, row])


__________________________________________________
Step 1
Completed Subtasks: 

Outdict: {'reason': 'Alice attempted to pick up Bread_1 in the previous step but was unsuccessful, possibly because she is not close enough to the object. Alice should navigate to Bread_1 to get close enough to successfully pick it up. Bob has just rotated and is currently looking at the underside of a countertop, which is not helpful for the current task. Bob needs to rotate again to have a better view of the environment and locate the fridge or the other objects (lettuce or tomato) he needs to interact with. A rotation to the right should bring more of the environment into view.', 'Alice': 'navigate to Bread_1', 'Bob': 'rotate in Right direction'}
